In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import glob

In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 2011
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:25:28
2025-02-17 01:25:59
2025-02-17 01:26:21
2025-02-17 01:26:46
2025-02-17 01:27:08
2025-02-17 01:27:33
2025-02-17 01:27:52
2025-02-17 01:28:09
2025-02-17 01:28:27
2025-02-17 01:28:46
2025-02-17 01:29:05
2025-02-17 01:29:27
2025-02-17 01:29:46
2025-02-17 01:30:06
2025-02-17 01:30:28
2025-02-17 01:30:49
2025-02-17 01:31:08
2025-02-17 01:31:31
2025-02-17 01:31:54
2025-02-17 01:32:24
2025-02-17 01:32:48
2025-02-17 01:33:15
2025-02-17 01:33:36
2025-02-17 01:34:01
2025-02-17 01:34:21
2025-02-17 01:34:51
2025-02-17 01:35:17
2025-02-17 01:35:31
2025-02-17 01:36:04
2025-02-17 01:36:25
2025-02-17 01:36:48
2025-02-17 01:37:11
2025-02-17 01:37:13
2025-02-17 01:37:31
2025-02-17 01:37:50
2025-02-17 01:38:13
2025-02-17 01:38:36
2025-02-17 01:39:09
2025-02-17 01:39:34
2025-02-17 01:39:54
2025-02-17 01:40:15
2025-02-17 01:40:36
2025-02-17 01:40:58
2025-02-17 01:41:24
2025-02-17 01:41:51
2025-02-17 01:42:13
2025-02-17 01:42:39
2025-02-17 01:43:05
2025-02-17 01:43:30
2025-02-17 01:43:56


In [8]:
import numpy as np
from datetime import datetime

def calculate_dn_averages(data, block_size=12):
    """
    计算每个时间块的平均温度值。

    参数:
    data -- 输入的温度数据数组。
    block_size -- 每个时间块的大小（默认为12个月）。

    返回:
    包含平均温度值的新数组，形状与输入数组相同。
    """
    # 创建一个形状相同的数组来存放结果，所有元素初始化为 NaN
    arr_t_dn = np.full_like(data, np.nan)
    
    # 循环处理每个位置的数据
    for i in range(data.shape[1]):

        for j in range(data.shape[2]):
            for k in range(data.shape[0] // block_size):
                # 选择第k个时间块的数据块进行平均
                arr_t_avg = np.mean(data[k*block_size:(k+1)*block_size, i, j])
                # 将平均结果填充到新数组的相应位置
                arr_t_dn[k*block_size:(k+1)*block_size, i, j] = arr_t_avg

    return arr_t_dn

# 使用示例：
# 假设 ds_selected 是你的数据集，并且已经加载了需要的数据。
# arr_t = ds_selected.t2.values
# 计算平均值
# result = calculate_monthly_averages(arr_t)


In [9]:
def cal_dn_t(array, dataset):
    lonn = dataset.lon.values
    latt = dataset.lat.values
    timee = dataset.time.values
    
    ds_dn_t = xr.Dataset(
        {'dnt': (['time', 'lat', 'lon'], array)},
        coords={
            'lon': (['lon'], lonn),
            'lat': (['lat'], latt),
            'time': ('time', timee)
        }
    )
    return ds_dn_t

In [8]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'
file_pattern_t = 'T2.wrf2d_d01_????-??-??.nc'
output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'

In [14]:
folder_names = [
   # 'U-50',
    'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]

In [15]:
for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path_t = os.path.join(base_path, folder, file_pattern_t)
    all_files_t = glob.glob(full_path_t)
    #####
    summer_files_t = [f for f in all_files_t if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f or '-05-' in f]
    ds = xr.open_mfdataset(summer_files_t)
    

    # 构建时间选择器，每年的5月31日18:00到9月1日05:00
    time_selection = (ds.time.dt.month == 5) & (ds.time.dt.day == 31) & (ds.time.dt.hour >= 18) | \
                     (ds.time.dt.month == 6) | \
                     (ds.time.dt.month == 7) | \
                     (ds.time.dt.month == 8) | \
                     ((ds.time.dt.month == 9) & (ds.time.dt.day == 1) & (ds.time.dt.hour <= 5))

    # 应用时间选择器
    ds_selected = ds.where(time_selection, drop=True)
    arr_t = ds_selected.t2.values
    arr_dnt = calculate_dn_averages(arr_t)
    ds_dnt = cal_dn_t(arr_dnt, ds_selected)
    ds_dnt_jja = ds_dnt.sel(time=ds_dnt['time'].dt.month.isin([6, 7, 8]))
    ds_dnt_jja.to_netcdf(output_folder+'dn_temp_'+folder+'.nc')


2024-12-27 16:13:58
2024-12-27 16:53:35
2024-12-27 17:32:56
2024-12-27 18:11:18
2024-12-27 18:48:48
2024-12-27 19:27:41
2024-12-27 20:05:14
2024-12-27 20:44:10


In [7]:
import numpy as np
from datetime import datetime

def calculate_dn_averages(data, block_size=12):
    """
    计算每个时间块的平均温度值。

    参数:
    data -- 输入的温度数据数组。
    block_size -- 每个时间块的大小（默认为12个月）。

    返回:
    包含平均温度值的新数组，形状与输入数组相同。
    """
    # 创建一个形状相同的数组来存放结果，所有元素初始化为 NaN
    arr_t_dn = np.full_like(data, np.nan)
    
    # 循环处理每个位置的数据
    for i in range(data.shape[1]):
        for j in range(data.shape[2]):
            for k in range(data.shape[0] // block_size):
                # 选择第k个时间块的数据块进行平均
                arr_t_avg = np.mean(data[k*block_size:(k+1)*block_size, i, j])
                # 将平均结果填充到新数组的相应位置
                arr_t_dn[k*block_size:(k+1)*block_size, i, j] = arr_t_avg

    return arr_t_dn

# 使用示例：
# 假设 ds_selected 是你的数据集，并且已经加载了需要的数据。
# arr_t = ds_selected.t2.values
# 计算平均值
# result = calculate_monthly_averages(arr_t)

def cal_dn_t(array, dataset):
    lonn = dataset.lon.values
    latt = dataset.lat.values
    timee = dataset.time.values
    
    ds_dn_t = xr.Dataset(
        {'dnt': (['time', 'lat', 'lon'], array)},
        coords={
            'lon': (['lon'], lonn),
            'lat': (['lat'], latt),
            'time': ('time', timee)
        }
    )
    return ds_dn_t

In [8]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'
file_pattern_t = 'TD2.wrf2d_d01_????-??-??.nc'
output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'

folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     #'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]

In [9]:
import glob
for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path_t = os.path.join(base_path, folder, file_pattern_t)
    all_files_t = glob.glob(full_path_t)
    #####
    summer_files_t = [f for f in all_files_t if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f or '-05-' in f]
    ds = xr.open_mfdataset(summer_files_t)
    

    # 构建时间选择器，每年的5月31日18:00到9月1日05:00
    time_selection = (ds.time.dt.month == 5) & (ds.time.dt.day == 31) & (ds.time.dt.hour >= 18) | \
                     (ds.time.dt.month == 6) | \
                     (ds.time.dt.month == 7) | \
                     (ds.time.dt.month == 8) | \
                     ((ds.time.dt.month == 9) & (ds.time.dt.day == 1) & (ds.time.dt.hour <= 5))

    # 应用时间选择器
    ds_selected = ds.where(time_selection, drop=True)
    arr_t = ds_selected.td2.values
    arr_dnt = calculate_dn_averages(arr_t)
    ds_dnt = cal_dn_t(arr_dnt, ds_selected)
    ds_dnt_jja = ds_dnt.sel(time=ds_dnt['time'].dt.month.isin([6, 7, 8]))
    ds_dnt_jja.to_netcdf(output_folder+'dn_dewtemp_'+folder+'.nc')

2024-12-29 00:48:59
2024-12-29 01:27:38
2024-12-29 02:05:22
2024-12-29 02:43:01
2024-12-29 03:19:52
2024-12-29 03:55:21
2024-12-29 04:30:53
2024-12-29 05:07:16
2024-12-29 05:43:24


In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# 假设有一个DataFrame名为df，包含'Income'和'Scores'两列
data = {
    'Income': np.random.normal(50000, 15000, 100),
    'Scores': np.random.normal(75, 10, 100) + np.random.normal(0, 0.05, 100) * np.random.normal(50000, 15000, 100)
}
df = pd.DataFrame(data)

# 分位数回归：分别对25%，50%，75%分位数进行回归
quantiles = [0.25, 0.5, 0.75]
models = {}
results = {}
for qt in quantiles:
    models[qt] = smf.quantreg('Scores ~ Income', df)
    results[qt] = models[qt].fit(q=qt)
    print(f'Results for {qt*100}% quantile:')
    print(results[qt].summary())


Results for 25.0% quantile:
                         QuantReg Regression Results                          
Dep. Variable:                 Scores   Pseudo R-squared:           -3.144e-05
Model:                       QuantReg   Bandwidth:                       1861.
Method:                 Least Squares   Sparsity:                        7538.
Date:                Sun, 12 Jan 2025   No. Observations:                  100
Time:                        15:15:23   Df Residuals:                       98
                                        Df Model:                            1
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1464.8099   1314.470     -1.114      0.268   -4073.333    1143.713
Income        -0.0075      0.025     -0.305      0.761      -0.056       0.041

The condition number is large, 2.19e+05. This might indicate that there are
strong multicollinearity o

/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
